# Indeed Job Scraper
Las siguientes secciones son explicación poco a poco del funcionamiento del código.
<br>Si se desea correr el scrapper completo dirijirse hasta la ultima sección del notebook


In [ ]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import random
import time

### Setup the query and url

In [ ]:
def get_url(position, location):
    # México URL
    #template = 'https://mx.indeed.com/jobs?q={}&l={}'
    
    # USA URL
    template = 'https://mx.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url

In [ ]:
url = get_url('ICT sales professional', 'Mexico')
print(url)


# Zen rows
Sen rows es una web scrapper api que me permite evitar bloqueos como los de Cloudflare mientras se hace el scrapping de datos.

In [ ]:
from zenrows import ZenRowsClient

client = ZenRowsClient("54aa66f34eb14f373da2382b7b152e9e05c89522")
url = "https://mx.indeed.com/jobs"

response = client.get(url)

print(response.text)

## Using Scrapper API

In [ ]:
url = "https://mx.indeed.com/rc/clk?jk=7910b2ddcbc5bd45&bb=jWqI-P_6WEJQJ_Ba2jRcqAJ8nzcvaHhy_w_1elTnf6KhnFh0ZUN5NkGxoEyAvmACJ-q8y2ul5-DES3NrFD7rTb021GLaJB4rYvShcQvMWAfOPXINI7Qz-waV9977zUmI&xkcb=SoBj67M3_ARg8hxSWp0LbzkdCdPP&fccid=4e7a4bb3f4e437e7&cmp=%25E8%258B%258F%25E5%25B7%259E%25E7%25AB%258B%25E8%25B7%2583%25E4%25BC%2581%25E4%25B8%259A%25E7%25AE%25A1%25E7%2590%2586%25E5%2592%25A8%25E8%25AF%25A2%25E6%259C%2589%25E9%2599%2590%25E5%2585%25AC%25E5%258F%25B8&vjs=3"
payload = { 'api_key': 'cef04c9c0f3b5be06796e360914d4955', 'url': url}
response = requests.get('https://api.scraperapi.com/', params=payload)


### Extract the html data

In [ ]:
# Añadiendo headers que simulan un navegador
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

soup = BeautifulSoup(response.text, 'html.parser')
print(response,"\n")
print(soup)

In [ ]:
session = requests.Session()  # Use session for connection pooling


In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
print(soup)

In [ ]:
# Usando la clase para encontrar todos los divs de la sección del trabajo
cards = soup.find_all('div', class_='job_seen_beacon')
print("Número de secciones encontradas:", len(cards))

### Prototype the model with a single record

In [ ]:
card = cards[0]

In [ ]:
job_title = card.h2.a.span.get('title')
print(job_title)


In [ ]:
company = card.find('span', {"data-testid":"company-name"}).text.strip()
print(company)

In [ ]:
job_location = card.find('div', {"data-testid":"text-location"}).text.strip()
print(job_location)

In [ ]:
post_date = card.find('span', {"data-testid":"myJobsStateDate"}).text.strip()
today = datetime.today().strftime('%Y-%m-%d')

print(post_date)

In [ ]:
summary = card.find('div', 'css-9446fg eu4oa1w0').text.strip().replace('\n', ' ')
print(summary)

In [ ]:
job_url = 'https://www.indeed.com' + card.h2.a.get('href')
print(job_url)

#### Job Details

In [ ]:
# response_Detail = requests.get(job_url, headers=headers)
# response_Detail = response

session = requests.Session()  # Use session for connection pooling
response_Detail = session.get(url)
soupDetail = BeautifulSoup(response_Detail.text, 'html.parser')
print(soupDetail)

In [ ]:
# this does not exists for all jobs, so handle the exceptions
salary_tag = soupDetail.find('div', 'js-match-insights-provider-tvvxwd')
if salary_tag:
    salary = salary_tag.text.strip()
else:
    salary = ''

print(salary)

In [ ]:
details = soupDetail.find('div', {"id":"jobDescriptionText"}).text.strip().replace('\n', ' ')
print(details)

In [ ]:
record = (job_title, company, job_location, post_date, today, summary, details,salary, job_url)

In [ ]:
record

### Generalize the model with a function

In [ ]:
def get_record(card, session, occ):
    job_title = card.h2.a.span.get('title')
    company = card.find('span', {"data-testid":"company-name"}).text.strip()
    job_location = card.find('div', {"data-testid":"text-location"}).text.strip()
    post_date = card.find('span', {"data-testid":"myJobsStateDate"}).text.strip()
    today = datetime.today().strftime('%Y-%m-%d')
    summary = card.find('div', 'css-9446fg eu4oa1w0').text.strip().replace('\n', ' ')
    job_url = 'https://www.indeed.com' + card.h2.a.get('href')

    response_Detail = session.get(job_url)
    soupDetail = BeautifulSoup(response_Detail.text, 'html.parser')

    salary_tag = soupDetail.find('div', 'js-match-insights-provider-tvvxwd')
    salary = salary_tag.text.strip() if salary_tag else ''

    details = soupDetail.find('div', {"id":"jobDescriptionText"}).text.strip().replace('\n', ' ') if soupDetail.find('div', {"id":"jobDescriptionText"}) else "No details provided"
    
    record = (occ, job_title, company, job_location, post_date, today, summary, details, salary, job_url)
    return record

In [ ]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)

### Safe request

In [ ]:
from requests.exceptions import ConnectionError, ChunkedEncodingError


def safe_request(url, session):
    """Keep trying the request until it succeeds."""
    while True:
        try:
            response = session.get(url)
            print("Respuesta de servidor: ", response)
            if response.status_code == 200:
                return response
            else:
                print(f"Respuesta no exitosa, código de estado: {response.status_code}. Reintentando...")
        except (ConnectionError, ChunkedEncodingError) as e:
            print(f"Error de conexión: {e}. Reintentando...")
        time.sleep(300)  # Espera 30 segundos antes de reintentar

### Get the next page

In [ ]:
while True:
    response = safe_request(url, session)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'job_seen_beacon')
    print("Número de secciones encontradas:", len(cards))
    if not cards:
        print("No se encontraron trabajos")
        break

    for card in cards:
        record = get_record(card, session, occ)
        writer.writerow(record)

    next_page = soup.find('a', {'data-testid': 'pagination-page-next'})
    if next_page:
        url = 'https://www.indeed.com' + next_page.get('href')
        print("Siguiente url: ", url)
    else:
        print("Se llego a la ultima página")
        break



***

### Using zenrows

In [ ]:
from requests.exceptions import ConnectionError, ChunkedEncodingError
from zenrows import ZenRowsClient
import time

session = ZenRowsClient("4d27f825dcf97f5ab5d203c7ddca2a24feb466ab")

def safe_request(url):
    """Keep trying the request until it succeeds."""
    while True:
        try:
            response = session.get(url)
            print("Respuesta de servidor: ", response)
            if response.status_code == 200:
                return response
            else:
                print(f"Respuesta no exitosa, código de estado: {response.status_code}. Reintentando...")
        except (ConnectionError, ChunkedEncodingError) as e:
            print(f"Error de conexión: {e}. Reintentando...")
            time.sleep(300)  # Espera 30 segundos antes de reintentar

In [ ]:
from requests.exceptions import ConnectionError, ChunkedEncodingError
from zenrows import ZenRowsClient
import time

session = ZenRowsClient("4d27f825dcf97f5ab5d203c7ddca2a24feb466ab")

def safe_request_detail(url):
    """Keep trying the request until it succeeds."""
    while True:
        try:
            response = session.get(url)
            print("Respuesta de servidor: ", response)
            if response.status_code == 200:
                return response
            else:
                print(f"Respuesta no exitosa, código de estado: {response.status_code}. Reintentando...")
        except (ConnectionError, ChunkedEncodingError) as e:
            print(f"Error de conexión: {e}. Reintentando...")
            time.sleep(300)  # Espera 30 segundos antes de reintentar

### Using ScrapperApi

In [1]:
import requests
import time

def safe_request(url):
    """Keep trying the request until it succeeds."""
    while True:
        try:
            payload = { 'api_key': 'cef04c9c0f3b5be06796e360914d4955', 'url': str(url) }
            response = requests.get('https://api.scraperapi.com/', params=payload)
            print("Respuesta de servidor: ", response)
            if response.status_code == 200:
                return response
            else:
                print(f"Respuesta no exitosa, código de estado: {response.status_code}. Reintentando...")
        except (ConnectionError, ChunkedEncodingError) as e:
            print(f"Error de conexión: {e}. Reintentando...")
            time.sleep(300)  # Espera 30 segundos antes de reintentar



In [2]:
import requests
import time

def safe_request_detail(url):
    """Keep trying the request until it succeeds."""
    while True:
        try:
            payload = { 'api_key': 'cef04c9c0f3b5be06796e360914d4955', 'url': str(url) }
            response = requests.get('https://api.scraperapi.com/', params=payload)
            if response.status_code == 200:
                return response
            else:
                print(f"Respuesta no exitosa, código de estado: {response.status_code}. Reintentando...")
        except (ConnectionError, ChunkedEncodingError) as e:
            print(f"Error de conexión: {e}. Reintentando...")
            time.sleep(300)  # Espera 30 segundos antes de reintentar

<div class="alert alert-block alert-warning">
<b>API KEYS:</b> Zen Rows y ScrapperAPI fueron de paga para permitir un scrapper más fluido
</div>

### Without web scrapping api

In [ ]:
import requests
import random
import time

def safe_request(url):
    """Keep trying the request until it succeeds."""
    while True:
        try:
            response = requests.get(url)
            print("Respuesta de servidor: ", response)
            print("HTML de servidor: ", response.content)
            if response.status_code == 200:
                return response
            else:
                print(f"Respuesta no exitosa, código de estado: {response.status_code}. Reintentando...")
        except (ConnectionError, ChunkedEncodingError) as e:
            print(f"Error de conexión: {e}. Reintentando...")
            time.sleep(300)  # Espera 30 segundos antes de reintentar


### Putting it all together

### ALL CODE

In [3]:
import csv
from datetime import datetime
from bs4 import BeautifulSoup

def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url

def get_record(card, occ):
    job_title = card.h2.a.span.get('title') if card.h2 and card.h2.a and card.h2.a.span else 'No title'
    company = card.find('span', {"data-testid":"company-name"}).text.strip() if card.find('span', {"data-testid":"company-name"}) else 'No company'
    job_location = card.find('div', {"data-testid":"text-location"}).text.strip() if card.find('div', {"data-testid":"text-location"}) else 'No location'
    post_date = card.find('span', {"data-testid":"myJobsStateDate"}).text.strip() if card.find('span', {"data-testid":"myJobsStateDate"}) else 'No date provided'
    today = datetime.today().strftime('%Y-%m-%d')
    summary = card.find('div', 'css-9446fg eu4oa1w0').text.strip().replace('\n', ' ') if card.find('div', 'css-9446fg eu4oa1w0') else 'No summary'
    job_url = 'https://www.indeed.com' + card.h2.a.get('href')

    response_Detail = safe_request_detail(job_url)
    soupDetail = BeautifulSoup(response_Detail.text, 'html.parser')

    salary_tag = soupDetail.find('div', 'js-match-insights-provider-tvvxwd')
    salary = salary_tag.text.strip() if salary_tag else ''

    details = soupDetail.find('div', {"id":"jobDescriptionText"}).text.strip().replace('\n', ' ') if soupDetail.find('div', {"id":"jobDescriptionText"}) else "No details provided"
    
    record = (occ, job_title, company, job_location, post_date, today, summary, details, salary, job_url)
    return record


def main(occ, location):
    url = get_url(occ, location)
    page_count = 0  # Inicializar el contador de páginas

    with open('indeed_scrapper/indeed_scrapper_June_SanFrancisco.csv', 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_ALL, escapechar='\\')
        # Check if the file is empty to write headers
        if f.tell() == 0:
            writer.writerow(['Occ', 'JobTitle', 'Company', 'Location', 'PostDate', 'ExtractDate', 'Summary', 'Details', 'Salary', 'JobUrl'])

        while page_count < 7:  # Límite de 7 páginas
            response = safe_request(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            cards = soup.find_all('div', 'job_seen_beacon')
            print("Número de secciones encontradas:", len(cards))
            if not cards:
                print("No se encontraron trabajos")
                break

            for card in cards:
                record = get_record(card, occ)
                writer.writerow(record)

            next_page = soup.find('a', {'data-testid': 'pagination-page-next'})
            if next_page:
                url = 'https://wwww.indeed.com' + next_page.get('href')
                page_count += 1
                print("Siguiente url: ", url)
            else:
                print("Se llego a la ultima página")
                break


In [4]:
occupations_list = [
    "ICT sales professional",
    "Marketing manager",
    "Product analyst",
    "Product manager",
    "Product designer",
    "Business Intelligence professional",
    "Infraestructure engineer",
    "Computer Systems Analyst",
    "Software infrastructure architect",
    "Web developer",
    "Software developer",
    "App developer",
    "User interface designer",
    "Software engineer",
    "Software architect",
    "Software Quality Assurance Analysts and Testers",
    "Embedded systems engineer",
    "Web and Digital Interface Designers",
    "Database infrastructure engineer",
    "Network architect",
    "Database Administrators",
    "Database Architects",
    "Network and Computer Systems Administrator",
    "Artificial intelligence engineer",
    "Machine Learning engineer",
    "Data science engineer",
    "Data analyst",
    "Data scientist",
    "Artificial Intelligence scientist",
    "Data architect",
    "ICT security specialist",
    "IT security operations",
    "Information Security Analysts",
    "Product Security and IT Security Integration Specialist",
    "Product risk specialist",
    "Security architect",
    "Database support engineer",
    "Data center operations engineer",
    "Support systems engineer",
    "Computer Network Support Specialists"
]


In [5]:
for occ in occupations_list:
    print(occ)
    main(occ, 'Mexico')

# run the main program

ICT sales professional
Respuesta de servidor:  <Response [200]>
Número de secciones encontradas: 1
Se llego a la ultima página
Marketing manager
Respuesta de servidor:  <Response [200]>
Número de secciones encontradas: 15
Siguiente url:  https://mx.indeed.com/jobs?q=Marketing+manager&l=Mexico&start=10
Respuesta de servidor:  <Response [200]>
Número de secciones encontradas: 15
Siguiente url:  https://mx.indeed.com/jobs?q=Marketing+manager&l=Mexico&start=20
Respuesta de servidor:  <Response [200]>
Número de secciones encontradas: 15
Siguiente url:  https://mx.indeed.com/jobs?q=Marketing+manager&l=Mexico&start=30
Respuesta de servidor:  <Response [200]>
Número de secciones encontradas: 15
Siguiente url:  https://mx.indeed.com/jobs?q=Marketing+manager&l=Mexico&start=40
Respuesta de servidor:  <Response [200]>
Número de secciones encontradas: 15
Siguiente url:  https://mx.indeed.com/jobs?q=Marketing+manager&l=Mexico&start=50
Respuesta de servidor:  <Response [200]>
Número de secciones enco